<a href="https://colab.research.google.com/github/Rishit605/AI-Art-generation/blob/main/Spactial_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Spactial Transformers** for CNNs and internalized augmented data adaptability


In [ ]:
# License: BSD
# Author: Ghassen Hamrouni

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
import glob
from tqdm import tqdm

plt.ion()   # interactive mode

In [ ]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

! pip install torchinfo
from torchinfo import summary

In [ ]:
!mkdir dataset
%cd dataset

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="BE0ifvf11oGilgCsqZFj")
project = rf.workspace("dylog").project("birds-detections")
dataset = project.version(4).download("yolov8")


/content/dataset/dataset
loading Roboflow workspace...
loading Roboflow project...
[WARNING] we noticed you are downloading a `yolov8` datasets but you don't have `ultralytics` installed. Roboflow `.deploy` supports only models trained with `ultralytics==8.0.134`, to intall it `pip install ultralytics==8.0.134`.


Extracting Dataset Version Zip to birds-detections-4 in yolov8:: 100%|██████████| 2974/2974 [00:00<00:00, 3706.19it/s]


In [ ]:
train_data = '/content/dataset/dataset/birds-detections-4/train/images'
valid_data = '/content/dataset/dataset/birds-detections-4/valid/images'

In [ ]:
## THIS IS OBSOLETE NEWER VERSION IS BELOW ##

class Img_fold(Dataset):
  def __init__(self, tar_dir, transform=None):
    self.paths = tar_dir
    self.transform = transform

  def load_image(self):
    return Image.open(self.paths)

  def __len__(self) -> int:
    for files in os.listdir(self.paths):
      item_path = os.path.join(self.paths, files)

      path_list = []

      if os.path.isfile(item_path) and item_path.endswith(('.jpg', '.png', '.jpeg')):
          path_list.append(item_path)
          return len(path_list)

  def __getitem__(self, idx):
    image = self.load_image(self.paths)

    if self.transform:
      return self.transform(image)
    else:
      return image

In [ ]:
imgs = cv2.imread('/content/dataset/Registration-Plate-3/train/images/frame227_jpg.rf.7034f4f9a4d92ec87716a7856cec1c15.jpg', 0)
imgs = cv2.resize(imgs, (128,128))
img_tensor = torch.from_numpy(imgs)
type(img_tensor)

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, img_dir: str):
    self.img_path = img_dir
    file_list = glob.glob(self.img_path + "*")
    # print(file_list)

    self.data=[]
    for class_path in file_list:
      class_name = "birds"
      for img_path in glob.glob(class_path + "/*.jpg"):
        self.data.append([img_path, class_name])
    # print(self.data)
    self.class_map = {"birds" : 0}
    self.img_dim = (128,128)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    img_path, class_name = self.data[idx]
    img = cv2.imread(img_path)
    img = cv2.resize(img, self.img_dim)
    # print(img.shape)

    class_id = self.class_map[class_name]
    img_tensor = torch.from_numpy(img)
    img_tensor = img_tensor.permute(2, 0, 1)
    class_id = torch.tensor([class_id],dtype=torch.float)
    return img_tensor.float(), class_id.float()

In [ ]:
data_aug = transforms.Compose([
    # transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.ToTensor()
])

In [ ]:
type(train_data)

str

In [ ]:
BATCH_SIZE = 4
NUM_WORKERS = os.cpu_count()
train_dataset = CustomDataset(train_data)

# Training dataset
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True,
                                           num_workers=NUM_WORKERS)

# Validation Dataset
valid_dataset = CustomDataset(valid_data)

valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=False,
                                           num_workers=NUM_WORKERS)

In [ ]:
train_loader

In [ ]:
for imgs, labels in train_loader:
    print("Batch of images has shape: ",imgs.shape, type(imgs))
    print("Batch of labels has shape: ", labels.shape, type(labels))

Batch of images has shape:  torch.Size([4, 3, 128, 128]) <class 'torch.Tensor'>
Batch of labels has shape:  torch.Size([4, 1]) <class 'torch.Tensor'>
Batch of images has shape:  torch.Size([4, 3, 128, 128]) <class 'torch.Tensor'>
Batch of labels has shape:  torch.Size([4, 1]) <class 'torch.Tensor'>
Batch of images has shape:  torch.Size([4, 3, 128, 128]) <class 'torch.Tensor'>
Batch of labels has shape:  torch.Size([4, 1]) <class 'torch.Tensor'>
Batch of images has shape:  torch.Size([4, 3, 128, 128]) <class 'torch.Tensor'>
Batch of labels has shape:  torch.Size([4, 1]) <class 'torch.Tensor'>
Batch of images has shape:  torch.Size([4, 3, 128, 128]) <class 'torch.Tensor'>
Batch of labels has shape:  torch.Size([4, 1]) <class 'torch.Tensor'>
Batch of images has shape:  torch.Size([4, 3, 128, 128]) <class 'torch.Tensor'>
Batch of labels has shape:  torch.Size([4, 1]) <class 'torch.Tensor'>
Batch of images has shape:  torch.Size([4, 3, 128, 128]) <class 'torch.Tensor'>
Batch of labels has 

In [ ]:
dataiter = iter(train_loader)
images, labels = next(dataiter)

In [ ]:
images.shape

torch.Size([4, 3, 128, 128])

In [ ]:
c1 = nn.Conv2d(3, 10, 5)
p1 = nn.MaxPool2d(2, 2)
d1 = nn.Dropout2d()
c2 = nn.Conv2d(10, 20, 5)

x = c1(images)
x = p1(x)
x = c2(x)
x = p1(x)
# x = d1(x)
x.shape

torch.Size([4, 20, 29, 29])

In [ ]:
## THis is for GrayScale Images

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

        # Spatial transformer localization-network
        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )

        # Regressor for the 3 * 2 affine matrix
        self.fc_loc = nn.Sequential(
            nn.Linear(10 * 3 * 3, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )

        # Initialize the weights/bias with identity transformation
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    # Spatial transformer network forward function
    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 10 * 3 * 3)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)

        return x

    def forward(self, x):
        # transform the input
        x = self.stn(x)

        # Perform the usual forward pass
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model = Net().to(device)

In [ ]:
summary(model, input_size=(4, 3, 128, 128))

In [ ]:
## For testing the Output Image shapes after Convolution
c1 = nn.Conv2d(3, 32, 7)
p1 = nn.MaxPool2d(2, 2)
d1 = nn.ReLU()
c2 = nn.Conv2d(32, 64, 5)

x = c1(images)
x = p1(x)
x = d1(x)
x = c2(x)
x = p1(x)
x = d1(x)
x.shape

torch.Size([4, 64, 28, 28])

In [ ]:
## This is for RGB images.

class STN(nn.Module):
    def __init__(self):
        super(STN, self).__init__()
        # simple convnet classifier
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32*29*29, 128)
        self.fc2 = nn.Linear(128, 84)
        self.fc3 = nn.Linear(84,1)

        # spatial transformer localization network
        self.localization = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=7),
            nn.MaxPool2d(2, 2),
            nn.ReLU(True),
            nn.Conv2d(32, 64, kernel_size=5),
            nn.MaxPool2d(2, 2),
            nn.ReLU(True),
        )
        # tranformation regressor for theta
        self.fc_loc = nn.Sequential(
            nn.Linear(64*28*28, 256),
            nn.ReLU(True),
            nn.Linear(256, 3 * 2)
        )
        # initializing the weights and biases with identity transformations
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0],
                                                    dtype=torch.float))
    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, xs.size(1)*xs.size(2)*xs.size(3))
        # calculate the transformation parameters theta
        theta = self.fc_loc(xs)
        # resize theta
        theta = theta.view(-1, 2, 3)
        # grid generator => transformation on parameters theta
        grid = F.affine_grid(theta, x.size())
        # grid sampling => applying the spatial transformations
        x = F.grid_sample(x, grid)
        return x
    def forward(self, x):
        # transform the input
        x = self.stn(x)

        # forward pass through the classifier
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32*29*29)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

model = STN().to(device)
summary(model, input_size=(4, 3, 128, 128))

Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.


Layer (type:depth-idx)                   Output Shape              Param #
STN                                      [4, 1]                    --
├─Sequential: 1-1                        [4, 64, 28, 28]           --
│    └─Conv2d: 2-1                       [4, 32, 122, 122]         4,736
│    └─MaxPool2d: 2-2                    [4, 32, 61, 61]           --
│    └─ReLU: 2-3                         [4, 32, 61, 61]           --
│    └─Conv2d: 2-4                       [4, 64, 57, 57]           51,264
│    └─MaxPool2d: 2-5                    [4, 64, 28, 28]           --
│    └─ReLU: 2-6                         [4, 64, 28, 28]           --
├─Sequential: 1-2                        [4, 6]                    --
│    └─Linear: 2-7                       [4, 256]                  12,845,312
│    └─ReLU: 2-8                         [4, 256]                  --
│    └─Linear: 2-9                       [4, 6]                    1,542
├─Conv2d: 1-3                            [4, 16, 124, 124]         

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

def train_step(model, dataloader,
              loss_fn, optimizer,
              device=device):

  model.train()

  train_loss, trian_acc = 0, 0

  for batch, (images, labels)  in enumerate(train_loader):
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()

    y_pred = model(images)

    # Convert labels to Long tensors
    # labels = labels.long()

    loss_fn = criterion(y_pred[0], labels)
    train_loss += loss_fn.item()

    loss_fn.backward()

    optimizer.step()

    _, preds = torch.max(y_pred.images, 1)
    train_acc += (preds == labels).sum().item()/len(y_pred)

  train_loss = train_loss/len(dataloader)
  train_acc = train_acc/len(dataloader)
  return train_loss, train_acc



def test_step(model, dataloader,
              loss_fn, optimizer,
              device=device):

  model.eval()

  test_loss, test_acc = 0, 0

  with torch.inference_mode():
    for batch, (images, labels) in enumerate(valid_loader):
      images, labels = images.to(device), labels.to(device)

      y_test_pred = model(images)

      loss_fn = criterion(y_test_pred[0], labels)
      test_loss +=  loss_fn.item()

      _, test_preds = torch.max(y_test_pred.images, 1)
      test_acc += (test_preds == labels).sum().item()/len(y_test_pred)

    test_loss = test_loss/len(dataloader)
    test_acc = test_acc/len(dataloader)
    return test_loss, test_acc


# def train_step(model, dataloader,
#               loss_fn, optimizer,
#               device=device):

#   model.train()

#   train_loss, trian_acc = 0, 0

#   for batch, (images, labels)  in enumerate(train_loader):
#     images, labels = images.to(device), labels.to(device)

#     optimizer.zero_grad()

#     y_pred = model(images)
#     image, logits = y_pred

  #   loss_fn = criterion(image, labels)
  #   train_loss += loss_fn.item()

  #   loss_fn.backward()

  #   optimizer.step()

  #   _, preds = torch.max(logits, 1)
  #   train_acc += (preds == labels).sum().item()/len(y_pred)

  # train_loss = train_loss/len(dataloader)
  # train_acc = train_acc/len(dataloader)
  # return train_loss, train_acc

# def test_step(model, dataloader,
#               loss_fn, optimizer,
#               device=device):

#   model.eval()

#   test_loss, test_acc = 0, 0

#   with torch.inference_mode():
#     for batch, (images, labels) in enumerate(dataloader):
#       images, labels = images.to(device), labels.to(device)

#       y_test_pred = model(images)
#       image, logits = y_test_pred

#       loss_fn = criterion(image, labels)
#       test_loss += loss_fn.item()

#       _, test_preds = torch.max(logits, 1)
#       test_acc += (test_preds == labels).sum().item()/len(y_test_pred)

#     test_loss = test_loss/len(dataloader)
#     test_acc = test_acc/len(dataloader)
#     return test_loss, test_acc


In [ ]:
def transform_to_numpy(image_grid, epoch):
    """
    This function transforms the PyTorch image grids
    into NumPy format that we will denormalize and save
    as PNG file.
    """
    image_grid = image_grid.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image_grid = std * image_grid + mean
    return image_grid

In [ ]:
images = []
def stn_grid(epoch):
    """
    This function will pass one batch of the test
    image to the STN model and get the transformed images
    after each epoch to save as PNG file and also as
    GIFFY file.
    """
    with torch.no_grad():
        data = next(iter(valid_loader))[0].to(device)
        transformed_image = model.stn(data).cpu().detach()
        image_grid = torchvision.utils.make_grid(transformed_image)
        # save the grid image
        image_grid = transform_to_numpy(image_grid, epoch)
        plt.imshow(image_grid)
        plt.savefig(f"../outputs/image_{epoch}.png")
        plt.close()
        images.append(image_grid)

In [ ]:
# # train for certain epochs

# epochs = 40

# for epoch in range(epochs):
#     print(f"Epoch {epoch+1} of {epochs}")

    # train_loss, train_acc = train_step(model, train_loader,
    #                                    optimizer, criterion)

#     test_loss, test_acc = test_step(model, valid_loader,
#                                     optimizer, criterion)
#     print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}")
#     print(f"Validation Loss: {test_loss:.4f}, Val Acc: {test_acc:.2f}")
#     stn_grid(epoch)

In [ ]:
train_loss, train_acc = train_step(model, train_loader,
                                       optimizer, criterion)

Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.


RuntimeError: ignored

## RGB Fin~

---

## GrayScale Cont-

In [ ]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='.', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), batch_size=64, shuffle=True, num_workers=4)
# Test dataset
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='.', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])), batch_size=64, shuffle=True, num_workers=4)

100%|██████████| 9912422/9912422 [00:00<00:00, 252270616.63it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 22407638.52it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 206007905.53it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 15822698.31it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw




This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.


In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
#
# A simple test procedure to measure the STN performances on MNIST.
#


def test():
    with torch.no_grad():
        model.eval()
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # sum up batch loss
            test_loss += F.nll_loss(output, target, size_average=False).item()
            # get the index of the max log-probability
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
              .format(test_loss, correct, len(test_loader.dataset),
                      100. * correct / len(test_loader.dataset)))

In [ ]:
def convert_image_np(inp):
    """Convert a Tensor to numpy image."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    return inp

# We want to visualize the output of the spatial transformers layer
# after the training, we visualize a batch of input images and
# the corresponding transformed batch using STN.


def visualize_stn():
    with torch.no_grad():
        # Get a batch of training data
        data = next(iter(test_loader))[0].to(device)

        input_tensor = data.cpu()
        transformed_input_tensor = model.stn(data).cpu()

        in_grid = convert_image_np(
            torchvision.utils.make_grid(input_tensor))

        out_grid = convert_image_np(
            torchvision.utils.make_grid(transformed_input_tensor))

        # Plot the results side-by-side
        f, axarr = plt.subplots(1, 2)
        axarr[0].imshow(in_grid)
        axarr[0].set_title('Dataset Images')

        axarr[1].imshow(out_grid)
        axarr[1].set_title('Transformed Images')

for epoch in range(1, 5 + 1):
    train(epoch)
    test()

# Visualize the STN transformation on some input batch
visualize_stn()

plt.ioff()
plt.show()

Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.


Train Epoch: 1 [0/1024 (0%)]	Loss: -0.000000


KeyboardInterrupt: ignored